## Exploratory Analysis

My exploratory analysis is characterized by a list of questions that I will try to argue in the next lines.

List of questions:

- $\textit{Which is the aim of the project?}$ 
The aim of the project is to develop a recommended algorithm to recommend movies to users.


- $\textit{Which data do we have?}$ 
We have the MovieLens dataset [1] (__[ml-20m](https://grouplens.org/datasets/movielens/20m/)__). It contains 20000263 ratings and 465564 tag applications across 27278 movies. These data were created by 138493 users between January 09, 1995 and March 31, 2015. This dataset was generated on October 17, 2016.


- $\textit{Is this dataset big?}$ 
The data is not "very" big, but it is quite large (553 Mb). Maybe, I have to use big data technologies to analyze it.


- $\textit{Have someone already studied this problem? Which is the bibliography? Have someone solved this problem?}$
Of course, I already know that recommended systems are studied from more than 10 years. Therefore, probably it exists several ways to solve this problems. This step is characterize by a bibliography study of recommended systems [2,3,4,5,6]. However, I can not spend many days studying it for obvious reasons. In fact, in these days the focus and the ability of the research is, in my opinion, very important. Fortunately, I already studied recommended systems and I know how to search.


- $\textit{From the basket of solution of the problem, how can I decide? Which method do I have to apply?}$ $\textit{Have I to develop a new method or not?}$
At this time, I already know what is user-user and item-item Collaborative Filtering, and Matrix Factorization. I already know that there are novel techniques using neuronal networks to solve the problem. Anyway, my decision is to use Matrix Factorization. My decision is stimulated by several reasons. First, the amount of data. The data are big, and also using Spark I prefer to have a result in a rapid time also considering the fact that this algorithm could be applied almost in real time recommended systems in future. Therefore I need to reduce the dimension of the matrix user-item. Matrix factorization [6] represents, in my opinion, the best option in this case. Maybe, there are methods using neuronal networks but I should study more to solve this problem from that point of view. 


- $\textit{How can I apply Matrix Factorization to this problem?}$
First, I start this point knowing perfectly the mathematics and the statistics that describe the factorization of the matrix user-item and its solution. I really believe that this step is very important for a Data Scientist and many times it is forgotten. This step is important because I have to know perfectly the mathematics to understand if that algorithm is the best solution. This step, sometimes, could take quite a long time (for example one or two days) because some mathematical solution are not trivial. Anyway, without this step there is no sense to try to apply an algorithm. If I do not have time to understand the algorithm it is better to downgrade and to apply a more simple algorithm. After the comprehension of the algorithm, I need to understand how to develop it, and there are two ways: write my own algorithm or use something already done (libriaries). There is no only one answer. Of course, I used to search and then I take my decision. Anyway the aim is to find/develop the best algorithm in the less time with the less number of code lines. In this case I already know the implementation of the Matrix Factorization for Collaborative Filtering in ApacheSpark using the Alternative Least Squares [7] to solve the system of equations. 

So... let's go to the algorithm!



#### Bibliography
-1. MovieLens 20M Dataset, https://grouplens.org/datasets/movielens/20m/

-2. https://medium.com/netflix-techblog/netflix-recommendations-beyond-the-5-stars-part-1-55838468f429

-3. https://medium.com/netflix-techblog/netflix-recommendations-beyond-the-5-stars-part-2-d9b96aa399f5

-4. Recommender Systems and Deep Learning in Python https://www.udemy.com/recommender-systems/

-5. Koren Y., Bell R., and Volinsky C. Matrix Factorization Techniques for Recommender Systems. IEEE Computer Society, (2009).

-6. Zhou Y., Wilkinson D., Schreiber R., and Pan R. Large-Scale Parallel Collaborative Filtering for the Netflix Prize. Springer Berlin Heidelberg, (2008).

-7. ApacheSpark ALS https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html


# Recommender System

## Introduction
The implementation of a recommender system algorithm is not a trivial problem to be solved for several reasons. The first reason is the dimension of the data that normally is used. In this case in the database ml-20m contains several files and among them we will use rating.csv. This file contains the list of ratings (from 1 to 5) made by each user (defined as userId) of each watched movie (defined as movieId). Also it is provided the timestamp that represent seconds since midnight Coordinated Universal Time (UTC) of January 1, 1970. Furthermore, there is a file called movies.csv that list all the movie with genre, however I decide to not use this file in this program.  
For the implementation of this program I will use only the file rating.csv (533.4 MB). As explained in the Explanatory Analysis I will use Apache Spark as cluster computing framework. Anyway to analyze the entire database we should use a web services (for example AWS). In this case, for obvious reasons, I will not run my program in a web service, but I decide to select only a part of the entire dataset to run my program locally. 

In [1]:
# Here I import all the libraries that I will use in the program
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import findspark 
findspark.init()
import pyspark
import random
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
import os
import csv
sc = pyspark.SparkContext(appName="Felipe_Recommender") # Here I initizalize the Spark Context

## 1-Step: Reading the file
In this step I read rating.csv file and I save it as RDD. As I explained in Introduction to run locally my program I can not use the entire database, therefore I have to choose which elements of the database I will use to train my model. It is important to always remember that using Spark, I can use this code for run the entire database with an account in a web service.

-How to choose the portion of the database? 
Actually, there exists many ways to choose the portion of the database. Maybe the simple way is randomly. Anyway in my case I decided to select the 10% more recent ratings. That is, in the next lines I order the database by timestamp from the more recent to the older and then I select the first 10% of elements (2000026).

In [2]:
# Reading the file
data = sc.textFile("DATA/ml-20m/ratings.csv") # Here I save the file as RDD
header = data.first() # Extraction of the header rembering that it is userId,movieId,rating,timestamp
data = data.filter(lambda row: row != header) # I remove the header of the file
data = data.mapPartitions(lambda x: csv.reader(x)) # I use csv reader to split each rows
data = data.sortBy(lambda a: a[3],ascending= False) # Here I sort the data according to the timestamp from the most recent to the older

In [3]:
data.take(5) # Here I simply test the impression of the first 5 rows after the sorting (It is like a test)

[['135350', '1193', '4.0', '999999978'],
 ['76737', '4376', '5.0', '999999896'],
 ['135350', '4506', '3.0', '999999827'],
 ['71975', '2045', '3.0', '999998906'],
 ['6388', '2402', '3.0', '999998716']]

In [4]:
n_rows = data.count() # Here simply I confirm that the number of rows is 20000263 (Test checked!)
n_rows

20000263

In [5]:
# Definition of the number of rows to analyze locally. Remeber that it is important to run locally the program
p_analyzed=0.1 # p_analyzed*100 is the percentage of the data anlyzed locally 
n_analyzed=int(n_rows*p_analyzed) # This is the number of the analyzed rows
n_analyzed

2000026

In [6]:
# Here I redefine the RDD data extracting only the first n_analyzed rows of the orginal database
data = data.zipWithIndex().filter(lambda i: i[1] < n_analyzed).keys()

## Step 2: Matrix Factorization
In this step I create the recommended system using Matrix Factorization. First I map my data into a Rating RDD and I create the train and the test set. I will train the model using the Alternative Least Square method [2-6]. 
The function train of ALS needs the ranking database (train), the number of feature for the factorization (k),
the number of iteration (n_iter), the value of lambda (lambda_) as regularization term, and a seed for randomly select the initial variables. The choice of the features k in my case is intuitive and also dictated by the computational power of my laptop. Anyway the features of the Matrix Factorization in this case are like the number movie genres. In this program I choose k=12. While lambda is the regularization term that it is used to balance the overfitting and the underfitting. It should be small, therefore I decide 0.01.

In [7]:
# Here I map my data in a Rating database made by user, product, and rating
ratings = data.map(  lambda l: Rating(int(l[0]), int(l[1]), float(l[2])))

In [8]:
""" In order to subsequently testing my recommender system I divide my reduced database
(that I called rating) in a training and test set. I decide 80% as training set. 
Of course, that value could be different, however "good" ranges are from 20% to 30%. 
"""
train, test = ratings.randomSplit([0.8, 0.2]) # Randomly create the training and the test set

In [9]:
""" Here I train the model (actually a solve the parametric equations with an iterative algorithm) 
using the Alternative Least Square method [2-6]. 
The function train of ALS needs the ranking database (train), the number of feature for the factorization (k),
the number of iteration, the value of lambda as regularization term, and a seed for randomly select
the initial variables. 
"""
K = 12  # Number of features
n_iter = 10 # Number of iteration for the algorithm. This algorithm  provides a numerical solution of a system of parametric equations and it is famous to converge rapidly.
lmb = 0.01 # Regularization factor
seed = 218721 # Seed for the initial conditions
model = ALS.train(train, K, iterations=n_iter,lambda_=lmb, seed = seed) # Trainig the Spark ALS model to the training set

In [10]:
################### THE MODEL IS TRAINED ############################################################

In [11]:
# Prediction! Here from the input of the user (_id_) my model provides as output the list of 10 movies
# to watch.
_id_=135350
model.recommendProducts(135350,10)

[Rating(user=135350, product=411, rating=8.16240751889596),
 Rating(user=135350, product=4209, rating=7.081661396393805),
 Rating(user=135350, product=4139, rating=6.650435640340947),
 Rating(user=135350, product=682, rating=6.613633434952787),
 Rating(user=135350, product=2963, rating=6.481643926261997),
 Rating(user=135350, product=4347, rating=6.450359976844619),
 Rating(user=135350, product=2960, rating=6.403528553066502),
 Rating(user=135350, product=4193, rating=6.260244784155489),
 Rating(user=135350, product=59, rating=6.1921494863334114),
 Rating(user=135350, product=2101, rating=6.158371905746923)]

## Step 3: Evaluation of the recommendation system
In this step I evaluate my recommender system. The choice of the evaluation method is, maybe, a more complex problem than the choice of the recommender algorithm. There exists many variation of evaluation measures [7]. Here, I decided to use the Mean Square Error (MSE). However it is important to underline the merits and the defects in the application of this method in movies recommendation. Starting from the merits, MSE is simple, widely used and it tends to penalize larger errors more severely than the other metrics [7]. However it is more appropriate for situations where we do not differentiate between errors. For movie recommendation, a rating equal to 2 is considerate negative, while a score equal to 3 is considered positive. It is like a threshold, therefore a mistake in the recommendation for these values is worst than for other values. Anyway knowing the merits and the defects, I believe that initially the best measure is the MSE for its simplicity and effectiveness. Probably in future I can develop other evaluation measures.

The MSE is defined as:
$\begin{equation}
MSE=\frac{1}{N}\sum_i^N (r_i-\hat{r}_i)^2
\end{equation}
$

Where $r_i$ is the real rate and $\hat{r}_i$ is the predicted rate of the tuple $i$. $N$ is the number of tuple user-movie.

In order to check if my model was trained in a fairly large database I calculate the MSE for the train and the test set. I hope to find the similar values and next to 0.

In [12]:
# MSE for the training set
x = train.map(lambda p: (p[0], p[1])) # Here I map the training set in x considering only user id and movie id
p = model.predictAll(x).map(lambda r: ((r[0], r[1]), r[2])) # Here I predict all the rating for the users and movies that are in x
ratesAndPreds = train.map(lambda r: ((r[0], r[1]), r[2])).join(p) # Here I join the result of the prediction with the real rates
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean() # Here I calculate Mean Square Error of the trainig set 
print("Training set MSE: "+ str(mse)) # Here I print the result for the training set 

Training set MSE: 0.5433501092014573


In [13]:
# MSE for the test set
x = test.map(lambda p: (p[0], p[1]))  # Here I map the test set in x considering only user id and movie id
p = model.predictAll(x).map(lambda r: ((r[0], r[1]), r[2]))  # Here I predict all the rating for the users and movies that are in x
ratesAndPreds = test.map(lambda r: ((r[0], r[1]), r[2])).join(p) # Here I join the result of the prediction with the real rates
mse = ratesAndPreds.map(lambda r: (r[1][0] - r[1][1])**2).mean() # Here I calculate Mean Square Error of the test set
print("Test mse: %s" % mse) # Here I print the result for the test set 

Test mse: 0.7946951606244492


## Conclusion

I obtained in the training set $MSE_{training}=0.54$ and in the test set $MSE_{test}=0.79$. The result for the training set is next to 0 (therefore it is good!), while for the test set is less good, however still quite small. An higher MSE for the test set is normal, anyway in order to reduce this difference there several ways: understand if there is an overfitting problem, find another ways to select the rating from the original database, or simply increasing the number of features. 

## Bibliography

-1. Recommender Systems and Deep Learning in Python https://www.udemy.com/recommender-systems/

-2. ApacheSpark ALS https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html and https://spark.apache.org/docs/latest/api/python/pyspark.ml.html#pyspark.ml.recommendation.ALS

-3. MatrixFactorizationModel https://spark.apache.org/docs/1.4.0/api/java/org/apache/spark/mllib/recommendation/MatrixFactorizationModel.html

-5. Koren Y., Bell R., and Volinsky C. Matrix Factorization Techniques for Recommender Systems. IEEE Computer Society, (2009).

-6. Zhou Y., Wilkinson D., Schreiber R., and Pan R. Large-Scale Parallel Collaborative Filtering for the Netflix Prize. Springer Berlin Heidelberg, (2008).

-7. Gunawardana A. and Shani G. A Survey of Accuracy Evaluation Metrics of Recommendation Tasks, (2009).